<a href="https://colab.research.google.com/github/KevinTheRainmaker/Recommendation_Algorithms/blob/main/colab/fastcampus/TF_IDF_Trial_with_Konlpy_mecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TF-IDF 실습

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 한국어 문장 분리 모듈
!pip install -q kss

     |████████████████████████████████| 42.4 MB 64 kB/s 
     |████████████████████████████████| 170 kB 67.1 MB/s 


In [3]:
# import packages
import os
import kss
import re

In [ ]:
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install Konlpy

# mecab
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

## Read Dataset
- news_sample.txt

### Functions for procesing

In [6]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

def cleansing_special(sentence):
    # 특수문자를 전처리를 하는 함수
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

def cleansing_numbers(sentence):
    # 숫자를 전처리(delexicalization) 하는 함수    
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence

In [7]:
from konlpy.tag import Mecab
mecab = Mecab()

### Retrieve Tags

In [9]:
data_path = '/content/drive/MyDrive/data'
all_sentences = []
with open(os.path.join(data_path, 'news_sample.txt'), 'r', encoding='utf-8') as f:
  for idx, line in enumerate(f.readlines()):
    print(f"--- 문서 {idx+1} 번 ---")
    preprocessed = cleansing_numbers(line)
    preprocessed = cleansing_special(preprocessed)
    # 명사만 추출하기
    preprocessed_news = ' '.join(list(set(mecab.nouns(preprocessed))))
    # 문장 분리해서 사용하는 방법
    # preprocessed_news = ' '.join(kss.split_sentences(preprocessed))
    print(preprocessed_news)
    all_sentences.append(preprocessed_news)

--- 문서 1 번 ---
공유 약점 국호 금강산 경협 질주 배치 퇴행 회한 인원 기초자치단체 처음 국익 지난해 후 꿈 몫 회생 남북한 도 화두 추진 햇살 행정 흑인 때 결정 암투 공감대 쇠 좌절 짐 인사 기반 구축 고유 절망 다음 불신 한반도 외교 시대 희망 정치력 슬기 청산 운행 나열 기대 근거 지혜 군 해다 문제 일 휘하 바람 유치 직무 고난 마음 속 열차 기 음모 미국 자치 번지 땅 올해 도약 련 시 혼란 글로벌 개성관광 한미 피 가치 유능 한뜻 생명력 단계 대화 부여 저력 떼 전력 성공 대결 전환점 년 추진력 간 소망 민 극복 지역 대통령 폐쇄 단체장 기축 듯 집행 수 대비 날 과제 태양 관행 현안 시간 증오 적재적소 목적 자질 후보 활용 올림픽 틀 생산 화해 표출 것 발전 도시 성향 바탕 불씨 몸 포함 역량 번 열쇠 예산 도민 시련 어제 한국 지방 역사 정과 단추 선진국 활력 이것 정치 단절 출발 관계 낭비성 동해 구체 자신 동기 방법 핵 과정 조화 욕 통일 국내 논리 암울 해결 기억 변화 통행 바다 다이 인구 도정 선거 협의 민족 그동안 지 인재 아침 단체 축년 최대 공직자 우리 이 기업 달성 발굴 인맥 정책 여건 코드 경제 당위 의 건국 개성 발휘 평화 국민 기회비용 성과 무엇 얼마 원년 자본 갈등 극대 등 강원도 행정력 미증유 육 청사진 유념 남북 중요 평창 정치색 당국 경의선 정부 정립 파악 조기 정상 동계 사무소 북 자생력 방향 실행 오바마 뒤 광채 능력 때문 모색 점 도전 미래 변수 사람 중단 면적 책임 위기 업그레이드 경기 현상 새해 취임
--- 문서 2 번 ---
요구 기관 부회장 사항 소 현장 개최 취약 걸 노력 지난해 교육청 꿈 운동장 월 도 일조 애 절차 폭 혼신 처장 기초 엘리트 생활 정기 성원 감사 구성원 전국 구축 반영 목별 연구 전폭 행복 진통 우수 자세 정관 희망 사무 확립 보람 운영 정호원 기원 기대 라인 밀알 제공 체전 군 어깨 도청 참여 전무 동물 일 융 기 터전 시스템 자치 올해 감회 사회 시 정체 마련 역동 메달

## TF-IDF using Scikit-learn

- 한글자 단어 제외
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
- Useful option: tokenizer, stop_words

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

In [11]:
print(len(all_sentences))

20


In [12]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
    word2id[feature] = idx

In [15]:
tfidf_vectorizer.vocabulary_

{'공유': 201,
 '약점': 1359,
 '국호': 301,
 '금강산': 335,
 '경협': 146,
 '질주': 2115,
 '배치': 876,
 '퇴행': 2331,
 '회한': 2610,
 '인원': 1718,
 '기초자치단체': 376,
 '처음': 2161,
 '국익': 294,
 '지난해': 2051,
 '회생': 2605,
 '남북한': 420,
 '화두': 2563,
 '추진': 2239,
 '햇살': 2501,
 '행정': 2509,
 '흑인': 2626,
 '결정': 122,
 '암투': 1344,
 '공감대': 188,
 '좌절': 1984,
 '인사': 1715,
 '기반': 355,
 '구축': 279,
 '고유': 183,
 '절망': 1891,
 '다음': 492,
 '불신': 987,
 '한반도': 2461,
 '외교': 1524,
 '시대': 1254,
 '희망': 2629,
 '정치력': 1935,
 '슬기': 1242,
 '청산': 2177,
 '운행': 1559,
 '나열': 409,
 '기대': 352,
 '근거': 328,
 '지혜': 2089,
 '해다': 2482,
 '문제': 768,
 '휘하': 2623,
 '바람': 812,
 '유치': 1617,
 '직무': 2095,
 '고난': 162,
 '마음': 681,
 '열차': 1454,
 '음모': 1628,
 '미국': 779,
 '자치': 1798,
 '번지': 884,
 '올해': 1515,
 '도약': 593,
 '혼란': 2554,
 '글로벌': 333,
 '개성관광': 77,
 '한미': 2460,
 '가치': 27,
 '유능': 1600,
 '한뜻': 2459,
 '생명력': 1094,
 '단계': 496,
 '대화': 571,
 '부여': 953,
 '저력': 1837,
 '전력': 1863,
 '성공': 1135,
 '대결': 531,
 '전환점': 1889,
 '추진력': 2240,
 '소망': 1170,
 '극복': 326,
 '지역'

In [16]:
tfidf_matrix.shape

(20, 2630)

In [17]:
with open(os.path.join(data_path, 'news_sample.txt'), 'r', encoding='utf-8') as f:
  for idx, line in enumerate(f.readlines()):
    print(f"--- 문서 {idx} 번 ---")
    print(line[:200])

--- 문서 0 번 ---
새로운 희망 공유하고 새 출발하자 기축년(己丑年) 새해다.새로운 희망을 공유하고 새 출발을 다짐할 때다.짙푸른 동해바다를 뚫고 이글거리는 광채를 뿜으며 힘차게 솟아오르는 태양을 안는다. 어제와 다른 이 아침의 햇살을 받으며 왜 간절한 소망이 없겠는가.지난날의 갈등과 혼란 회한을 뒤로하고 새로 일어서야 한다.올해는 불신과 증오, 암투와 음모, 좌절과 절망 등
--- 문서 1 번 ---
<새해 이렇게 하겠습니다>스포츠 "최일선 현장에서 진솔한 소리 듣겠다" 최동용 도체육회 사무처장 지도자와 선수 등 체육의 최일선 현장에서 진솔한 소리를 더 많이 듣겠다. 강원체육이 지금까지 잘 할 수 있었던 것은 운동장 체육관 등 현장에서 지도자와 선수들이 묵묵히 운동한 결과다. 애로사항 등을 청취 체육정책에 반영하는 소통의 역할에 충실하겠다. 도청·도교육
--- 문서 2 번 ---
<새해 이렇게 하겠습니다>경제 '저탄소 녹색성장' 도모 전수산 도상공회의소협의회장 강원경제는 경제성장에 필요한 입지여건이나 기반조성이 타 지역보다 열악하고 최근 세계경제 침체의 영향으로 많은 어려움을 겪고 있다. 그러나 최근 수도권과의 교통망이 확충되면서 중앙 및 지방정부의 지역전략산업 육성 등 유리한 발전여건을 갖춰 나가고 있다. 정부의 5+2광역경제권 
--- 문서 3 번 ---
<새해 이렇게 하겠습니다>문화 이대근 강원도문화원연합회장 “지속적 향토사 연구로 우리 역사 되살리겠다” 어김없이 새로운 한 해가 밝았다. 지난해는 많은 시련을 안겨줬다. 선진국의 문턱에서 넘지 못하고 머뭇거리며 지나온 10년 세월이 원망스러운 가운데 이번에는 미국발 경제한파가 발목을 잡고 있다. 우리나라는 국민소득에서 보면 전 세계에서 24번째 국가에 머물
--- 문서 4 번 ---
<새해 이렇게 하겠습니다>속초 고성 양양 ■속초시 환동해 수출입항 메카 거듭 의정 옴부즈맨 제도 활성화 채용생 속초시장은 “올해는 민선4기를 사실상 마무리 짓는 해로 지금까지 성과를 바탕으로 성실한 마무리와 함께 경제

In [18]:
for idx, sent in enumerate(all_sentences):
    print(f"--- 문서 {idx} 번 ---")
    results = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
    results.sort(key = lambda element : element[1], reverse=True)
    print(results)
    print("\n")

--- 문서 0 번 ---
[('약점', 0.08316005260563082), ('국호', 0.08316005260563082), ('경협', 0.08316005260563082), ('질주', 0.08316005260563082), ('퇴행', 0.08316005260563082), ('회한', 0.08316005260563082), ('기초자치단체', 0.08316005260563082), ('국익', 0.08316005260563082), ('남북한', 0.08316005260563082), ('화두', 0.08316005260563082), ('햇살', 0.08316005260563082), ('흑인', 0.08316005260563082), ('암투', 0.08316005260563082), ('공감대', 0.08316005260563082), ('불신', 0.08316005260563082), ('한반도', 0.08316005260563082), ('외교', 0.08316005260563082), ('정치력', 0.08316005260563082), ('청산', 0.08316005260563082), ('운행', 0.08316005260563082), ('나열', 0.08316005260563082), ('근거', 0.08316005260563082), ('해다', 0.08316005260563082), ('휘하', 0.08316005260563082), ('직무', 0.08316005260563082), ('열차', 0.08316005260563082), ('음모', 0.08316005260563082), ('번지', 0.08316005260563082), ('혼란', 0.08316005260563082), ('개성관광', 0.08316005260563082), ('한미', 0.08316005260563082), ('유능', 0.08316005260563082), ('생명력', 0.08316005260563082), ('부여', 0.0831600